# Tutorial for AFMpy.REC.DSC

## Imports

In [ ]:
# Standard library imports
import json
import logging

# Third party imports
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import Stack, DL, Plotting, REC

## Configure Logging

In [ ]:
# Load the preconfigured logging settings
with open('logs/DSC_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Config

In [ ]:
Plotting.configure_formatting()

## Load the Stacks

In [ ]:
# Set the filepath for the public key to verify the integrity of the stacks.
PUBLIC_KEY_FILEPATH = '../common/keys/Tutorial_Public.pub'

# Load the cytoplasmic and periplasmic stacks
cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AC-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AP-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

In [ ]:
plt.imshow(cytoplasmic_stack.stack[0], cmap = Plotting.LAFMcmap)

## Prepare the Convolutional Autoencoders

In [ ]:
# Determine the input shape for the CAE. It should be (width,height,channels). In our case (64,64,1)
cyto_input_shape = (*cytoplasmic_stack.stack.shape[1:], 1)
peri_input_shape = (*periplasmic_stack.stack.shape[1:], 1)

# Create the Convolutional Autoencoder models to train with our data.
cytoplasmic_CAE = DL.DefaultCAE(input_shape = cyto_input_shape)
periplasmic_CAE = DL.DefaultCAE(input_shape = peri_input_shape)

## Apply Deep Spectral Clustering

### Cytoplasmic

#### Train the model and apply spectral clustering

In [ ]:
# Use DSC to determine the cluster labels for the cytoplasmic stack  with n=4 clusters
cyto_clusters = REC.DSC(cytoplasmic_stack, cytoplasmic_CAE, n_clusters = 4)

#### Generate the LAFM Images for each cytoplamic cluster

In [ ]:
# Generate the mean and LAFM images for each cluster.
for cluster in cyto_clusters:
    cluster.calc_mean_image()
    cluster.calc_LAFM_image(target_resolution = (96,96), sigma = 2.25)

#### Display the cytoplasmic clustered LAFM images

In [ ]:
fig, ax = plt.subplots(4,2, figsize = (6,12))
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])
ax[0,0].imshow(cyto_clusters[0].mean_image, cmap = Plotting.LAFMcmap)
ax[0,1].imshow(cyto_clusters[0].LAFM_image, cmap = Plotting.LAFMcmap)
ax[1,0].imshow(cyto_clusters[1].mean_image, cmap = Plotting.LAFMcmap)
ax[1,1].imshow(cyto_clusters[1].LAFM_image, cmap = Plotting.LAFMcmap)
ax[2,0].imshow(cyto_clusters[2].mean_image, cmap = Plotting.LAFMcmap)
ax[2,1].imshow(cyto_clusters[2].LAFM_image, cmap = Plotting.LAFMcmap)
ax[3,0].imshow(cyto_clusters[3].mean_image, cmap = Plotting.LAFMcmap)
ax[3,1].imshow(cyto_clusters[3].LAFM_image, cmap = Plotting.LAFMcmap)

### Periplasmic

#### Train the model and apply spectral clustering

In [ ]:
# Use DSC to determine the cluster labels for the periplasmic stack  with n=2 clusters
peri_clusters = REC.DSC(periplasmic_stack, periplasmic_CAE, n_clusters = 2)

#### Generate the LAFM Images for each periplamic cluster

In [ ]:
# Generate the mean and LAFM images for each cluster.
for cluster in peri_clusters:
    cluster.calc_mean_image()
    cluster.calc_LAFM_image(target_resolution = (96,96), sigma = 2.25)

#### Display the periplasmic clustered LAFM images

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (6,6))
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])
ax[0,0].imshow(peri_clusters[0].mean_image, cmap = Plotting.LAFMcmap)
ax[0,1].imshow(peri_clusters[0].LAFM_image, cmap = Plotting.LAFMcmap)
ax[1,0].imshow(peri_clusters[1].mean_image, cmap = Plotting.LAFMcmap)
ax[1,1].imshow(peri_clusters[1].LAFM_image, cmap = Plotting.LAFMcmap)